In [1]:
# JupyterNotebook Imports
from jupyter_plotly_dash import JupyterDash

# Dash Imports
import dash
import dash_table
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Data manipulation imports
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from pymongo import MongoClient


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password, "localhost", 52170, "AAC")

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read(projection={'_id': False}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SammyShuck_ModuleFive')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Animal Shelter - Sammy Shuck'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        # format the table so that the columns adjust automatically absed on the cell values
        style_header={
            'whiteSpace': 'normal',
            'height': 'auto',
        },
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,                  # disallow editing
        filter_action="native",          # allow native column filtering
        sort_action="native",            # allow column sorting
        sort_mode="multi",               # allow multiple columns to be sorted
        column_selectable=True,          # allow multiple columns to be selected
        row_selectable=False,            # disallow rows being selected
        row_deletable=False,             # disallow rows being deleted
        selected_columns=[],
        selected_rows=[],
        page_action="native",            # enable pagination
        page_current=0,                  # start at the first page on load
        page_size=10,                    # set the number of rows per page to 10

    ),
    html.Br(),
    html.Hr(),
    # create a div for the map data
    html.Div(id='map-id', className='col s12 m6')
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#d2ffd7' # changed the color to a green instead of blue
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):

    # retrive the pandas dataFrame for the view data
    dff = pd.DataFrame.from_dict(viewData)

    # initialize the marker list, defining the tile layer
    markers = [dl.TileLayer(id="base-layer-id")]

    # iterating the DataFrame rows is a long process. In order to improve performance
    #  let's iterate over the index and and reference each row by the index value to
    #  retrieve the column data.
    # Create a dl.Marker for every item in the viewData so that multiple markers
    #  will be shown at once
    for index in dff.index:
        # extend is more efficient than concatenation +=
        markers.extend([dl.Marker(position=[dff["location_lat"][index], dff["location_long"][index]],
                              children=[
                                  dl.Tooltip(dff["name"][index] if dff["name"][index] else "Not Named"),
                                  dl.Popup([
                                      html.H2(f"Name: {dff['name'][index] if dff['name'][index] else 'Not Named'}"),
                                      html.P(f"Type: {dff['animal_type'][index]}"),
                                      html.P(f"Breed: {dff['breed'][index]}"),
                                      html.P(f"Color: {dff['color'][index]}"),
                                      html.P(f"Sex: {dff['sex_upon_outcome'][index]}")
                                  ])
                              ]
                              )
                    ])
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               # center the geo map so that all points are visible
               center=[
                   (np.max(dff['location_lat'].to_numpy()) + np.min(dff['location_lat'].to_numpy())) / 2.0,
                   (np.max(dff['location_long'].to_numpy()) + np.min(dff['location_long'].to_numpy())) / 2.0
               ],
               zoom=10,
               # if wanting to show just the first Dog entry use children=markers[:2]
               # if wanting to show ALL Dogs int he current viewable table use children=markers
               children=markers[:2]
               )
    ]

app

ModuleNotFoundError: No module named 'jupyter_plotly_dash'